In [1]:
import os
import openai

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [22]:
import pandas as pd

In [106]:
# Load existing or create new dataframe for results
path_to_df_results = 'df_results.pkl'
if os.path.exists(path_to_df_results):
    df_results = pd.read_pickle(path_to_df_results)
else:
    print("df_results not found.\nEmpty df_results was created")
    df_results = pd.DataFrame(columns=[
        'id', 'model', 'prompt', 'text_response', 'finish_reason', 'temperature', 'max_tokens', 'top_p', 'frequency_penalty', 'presence_penalty', 'prompt_tokens', 'completion_tokens', 'total_tokens', 'approx_price_usd'
    ])
    
df_results

df_results not found.
Empty df_results was created


,id,model,prompt,text_response,finish_reason,temperature,max_tokens,top_p,frequency_penalty,presence_penalty,prompt_tokens,completion_tokens,total_tokens,approx_price_usd


In [62]:
# General Settings
model="text-davinci-003"
temperature=0.5
max_tokens=60
top_p=1.0
frequency_penalty=0.8
presence_penalty=0.0

prompts_list = [
    "Extract keywords from this text:\n\nBlack-on-black ware is a 20th- and 21st-century pottery tradition developed by the Puebloan Native American ceramic artists in Northern New Mexico. Traditional reduction-fired blackware has been made for centuries by pueblo artists. Black-on-black ware of the past century is produced with a smooth surface, with the designs applied through selective burnishing or the application of refractory slip. Another style involves carving or incising designs and selectively polishing the raised areas. For generations several families from Kha'po Owingeh and P'ohwhóge Owingeh pueblos have been making black-on-black ware with the techniques passed down from matriarch potters. Artists from other pueblos have also produced black-on-black ware. Several contemporary artists have created works honoring the pottery of their ancestors.",
    "Extract keywords from this text:\n\nCould you please generate various graphics of a coat of arms/symbol for the Elven House of Ulthuan in the Warhammer Fantasy RPG world, based on the following description: ""A feather and lightning bolt above a silver bird - the feather symbolizes wisdom and knowledge, the lightnining bolt denotes valor and power, and the silver bird is a representation of spiritual transformation. Together, they embody knowledge that leads to the fight for wisdom."" Please create unique and visually appealing designs for this coat of arms/symbol. Thank you.",
    "Extract keywords from this text:\n\ncoat of arms/symbol for the Elven House of Ulthuan in the Warhammer Fantasy RPG world. Coat contains: a feather and lightning bolt above a silver bird - the feather symbolizes wisdom and knowledge, the lightnining bolt denotes valor and power, and the silver bird is a representation of spiritual transformation. Together, they embody knowledge that leads to the fight for wisdom.",

]

model_cost_per_1k = 0.0200
total_tokens_all_prompts = 0
total_cost_for_all_prompts = 0

for index, prompt in enumerate(prompts_list):
    print(f"Executing for {index:3} of {len(prompts_list):3}.\t Total tokens used: {total_tokens_all_prompts:10}. \t Approx_total_cost: {total_cost_for_all_prompts}", end='\r')

    # Get response
    response = openai.Completion.create(
        model=model, 
        prompt=prompt, 
        temperature=temperature, 
        max_tokens=max_tokens, 
        top_p=top_p, 
        frequency_penalty=frequency_penalty, 
        presence_penalty=presence_penalty
    )
    
    # Calculate approx price and update total values
    total_tokens = response.usage.total_tokens
    approx_price_usd = (total_tokens * model_cost_per_1k)/1000
    total_tokens_all_prompts += total_tokens
    total_cost_for_all_prompts += approx_price_usd

    # Add row to dataframe
    df_row = pd.DataFrame([
        {
            'id': response.id,
            'model': model,
            'prompt': prompt,
            'finish_reason': response.choices[0].finish_reason,
            'text_response': response.choices[0].text,
            'temperature': temperature,
            'max_tokens': max_tokens,
            'top_p': top_p,
            'frequency_penalty': frequency_penalty,
            'presence_penalty': presence_penalty,
            'prompt_tokens': response.usage.prompt_tokens,
            'completion_tokens': response.usage.completion_tokens,
            'total_tokens': response.usage.total_tokens,
            'approx_price_usd': approx_price_usd
        }
    ])
    df_results = pd.concat([df_results, df_row], ignore_index=True)

In [72]:
df_results.to_pickle('df_results.pkl')

In [71]:
df_results

,id,model,prompt,text_response,finish_reason,temperature,max_tokens,top_p,frequency_penalty,presence_penalty,prompt_tokens,completion_tokens,total_tokens,approx_price_usd
0,cmpl-7Behq1FGoALRE4N3AfaAl87mixLXT,text-davinci-003,Extract keywords from this text:\n\nBlack-on-b...,"\n\nKeywords: Black-on-black ware, 20th centur...",length,0.5,60,1.0,0.8,0.0,186,60,246,0.00492
1,cmpl-7Behu6YzhRqnYMnu7DJgsEwizh3wz,text-davinci-003,Extract keywords from this text:\n\nCould you ...,"\n\nKeywords: coat of arms, symbol, Elven Hous...",stop,0.5,60,1.0,0.8,0.0,116,42,158,0.00316
2,cmpl-7BehyggkyiolmMjOp1oUl0qz8KVIO,text-davinci-003,Extract keywords from this text:\n\ncoat of ar...,\n\n- Coat of Arms \n- Symbol \n- Elven House ...,length,0.5,60,1.0,0.8,0.0,86,60,146,0.00292


In [70]:
import textwrap
# Extract the value of the 'text' field
text = df_results.iloc[1]['text_response']

# Print the text with line breaks / word wrap
wrapped_text = textwrap.fill(text, width=120)
print(wrapped_text)

  Keywords: coat of arms, symbol, Elven House of Ulthuan, Warhammer Fantasy RPG world, feather, lightning bolt, silver
bird, wisdom, knowledge, valor, power, spiritual transformation


In [105]:
for index, row in df_results.iterrows():
    print(f"{'▬'*35} start: {index:3}   {'▬'*115}")
    print(f"PROMPT: \n{row.prompt}\n")
    print("-"*165)
    print("\nTEXT_RESPONSE:")
    print(textwrap.fill(row['text_response'], width=120))
    print(f"{'▬'*35}   end: {index:3}   {'▬'*115}")
    # print("▬"*150)
    print("\n\n")

▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬ start:   0   ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
PROMPT: 
Extract keywords from this text:

Black-on-black ware is a 20th- and 21st-century pottery tradition developed by the Puebloan Native American ceramic artists in Northern New Mexico. Traditional reduction-fired blackware has been made for centuries by pueblo artists. Black-on-black ware of the past century is produced with a smooth surface, with the designs applied through selective burnishing or the application of refractory slip. Another style involves carving or incising designs and selectively polishing the raised areas. For generations several families from Kha'po Owingeh and P'ohwhóge Owingeh pueblos have been making black-on-black ware with the techniques passed down from matriarch potters. Artists from other pueblos have also produced black-on-black ware. Several contemporary artists have created works honor

In [ ]:
# response = openai.Completion.create(
#     model="text-davinci-003",
#     prompt="Extract keywords from this text:\n\nBlack-on-black ware is a 20th- and 21st-century pottery tradition developed by the Puebloan Native American ceramic artists in Northern New Mexico. Traditional reduction-fired blackware has been made for centuries by pueblo artists. Black-on-black ware of the past century is produced with a smooth surface, with the designs applied through selective burnishing or the application of refractory slip. Another style involves carving or incising designs and selectively polishing the raised areas. For generations several families from Kha'po Owingeh and P'ohwhóge Owingeh pueblos have been making black-on-black ware with the techniques passed down from matriarch potters. Artists from other pueblos have also produced black-on-black ware. Several contemporary artists have created works honoring the pottery of their ancestors.",
#     temperature=0.5,
#     max_tokens=60,
#     top_p=1.0,
#     frequency_penalty=0.8,
#     presence_penalty=0.0
# )